<a href="https://colab.research.google.com/github/Cseudave/materias_universitarias/blob/main/MisProfesores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Librerias

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random

In [ ]:
import pandas as pd
import json

## Estructura de la página

1. La lista de universidades lista en 71 páginas las diferentes escuelas de México de las que lso estudiantes han calificado a sus profesores. 
  - Sin embargo, hay muchas universidades duplicadas.
  
  Nos interesa:
  *   Link de universidad
  *   Ciudad
  *   Estado


2. Cada universidad tiene su página tiene un listado de los profesores registrados. 
  - Algunos profesores tienen doble registro porque tienen más de una materia o porque han sido registrados con su nombre y después su apodo.
3. Cada profesor tiene una página que lista los comentarios sobre él o ella, donde nos interesa:

  Nos interesa:
  - Materia impartidas
  - Calidad general
  - Nivel de recomendación general
  - Nivel de dificultad general

  - Comentarios
      - Materia impartida
      - Calidad general
      - Facilidad

    Solo si cuenta con una o más reseñas

## Obtener lista de universidades

In [ ]:
url = 'https://www.misprofesores.com/Universidades'

### Obtener todas las paginas de la lista de universidades

In [ ]:
## Obtener lista de urls de las listas de universidades

response = requests.get(url)

if response.status_code == 200:
  soup = BeautifulSoup(response.content, 'html.parser')
all_pages = soup.find('nav', {'aria-label':"Page navigation"})
pages = all_pages.find_all('li')

url_base = 'https://www.misprofesores.com/Universidades'
all_url = []

for page in pages:
  all_url.append(url_base + str(page.find('a').get('href')))
all_url.pop(0)


'https://www.misprofesores.com/Universidades#'

### Obtener datos de las universidades de una pagina

In [ ]:
def get_universidades(url):
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    rows = soup.find_all('table', class_='main')
    for row in rows:
      cells = row.find_all('td')

    universidad = []
    url_universidad = []
    ciudad = []
    estado = []
    n_profesores = []

    n = (len(cells)-2)//4
    for i in range(0, n):

      universidad.append(cells[2+4*i].text)
      url_universidad.append(cells[2+4*i].find('a').get('href'))
      ciudad.append(cells[3+4*i].text)
      estado.append(cells[4+4*i].text)
      n_profesores.append(cells[5+4*i].text)

    df = pd.DataFrame({'Universidad': universidad,
                  'url_universidad': url_universidad,
                  'ciudad': ciudad,
                  'estado': estado, 
                  'n_profesores':n_profesores})
    return df

  else:
    print('Error', response.status_code)
    print(url)
    return 'Error'


In [ ]:
test_url = all_url[0:3]

## Obtener urls de universidades

In [ ]:
for url in all_url:
# for url in test_url:
  df_temp = get_universidades(url)
  try:
    df_general = pd.concat([df_general, df_temp], ignore_index=True) 
  except:
    df_general = df_temp  
  time.sleep(1)


In [ ]:
df_general.to_csv('All_universidades.csv', index=False)

In [ ]:
df_general = pd.read_csv('All_universidades.csv')

In [ ]:
df_general.head()

,Universidad,url_universidad,ciudad,estado,n_profesores
0,"!""-ESIQIE,-IPN""",https://www.misprofesores.com/escuelas/ESIQIEI...,cd de mexico.,CD DE MEXICO,48
1,"""-ESIQIE,-IPN""",https://www.misprofesores.com/escuelas/ESIQIEI...,CD DE MEXICO,MEXICO,554
2,"""-ESIQIE,-IPN"".",https://www.misprofesores.com/escuelas/ESIQIEI...,CD DE MEXICO,MEXICO,51
3,"""-ESIQIE,IPN""",https://www.misprofesores.com/escuelas/ESIQIEI...,MÉXICO,CD DE MÉXICO,723
4,"""-ESIQIE,IPN.""",https://www.misprofesores.com/escuelas/ESIQIEI...,D.F.,MÉXICO,135


Se podría obtener los datos de cada universidad, sin embargo hay muchas entradas repetidas o mal escritas. La mayoría no tienen muchos profesores, lo que no tiene sentido si son escuelas funcionando.
 - Una posibilidad es que los estudiantes no han registrado muchos profesores
 - La entrada no es una universidad valida
 - Quien hizo el registró una reseña para una universidad ya registrada

Entonces, eliminaremos las universidades que muy pocos valores

In [ ]:
# Sin embargo primero debo convertir los datos a un tipo más adecuado
df_general.dtypes

Universidad        object
url_universidad    object
ciudad             object
estado             object
n_profesores        int64
dtype: object

In [ ]:
df_general['n_profesores'] = df_general['n_profesores'].astype(int)

In [ ]:
df_general.dtypes

Universidad        object
url_universidad    object
ciudad             object
estado             object
n_profesores        int64
dtype: object

In [ ]:
# Promedio
description = df_general['n_profesores'].describe()
median = df_general['n_profesores'].median()

In [ ]:
print(median)

2.0


Después de revisar la muchas entradas de universidades con solo una o dos reseñas, he notado que no tienen calificaciones, por lo que no son relevantes. En otros casos, son bromas o escuelas inexistentes

In [ ]:
df_general = df_general.drop(df_general.loc[df_general['n_profesores']<=2].index)

In [ ]:
df_general['n_profesores'].describe()

count    2955.000000
mean       50.614552
std       208.951760
min         3.000000
25%         4.000000
50%         7.000000
75%        22.000000
max      3728.000000
Name: n_profesores, dtype: float64

Donde es razonable una planta promedio de casi 40 profesores. Aunque el ideal sería que el primer cuartil fuera de mínimo 20 profesores. Sin embargo, como el proyecto pretende agrupar las entradas de universidades repetidas, por ahora consideraré la mayor cantidad de universidades

In [ ]:
df_general.to_csv('uni_validas.csv', index=False)

## Obtener comentarios de los proresores de la Facultad de ciencias

In [ ]:
df_general = pd.read_csv('uni_validas.csv') 

In [ ]:
df_general.sort_values(by='n_profesores', ascending=False).head()

,Universidad,url_universidad,ciudad,estado,n_profesores
1777,Tec de Monterrey (ITESM - MTY),https://www.misprofesores.com/escuelas/Tec-de-...,Monterrey,Nuevo León,3728
2117,UNAM,https://www.misprofesores.com/escuelas/UNAM_1059,México,D.F.,3657
966,Facultad de Ciencias UNAM,https://www.misprofesores.com/escuelas/Faculta...,Ciudad de Mexico,Distrito Federal,3587
140,BUAP - Benemérita Universidad Autónoma de Puebla,https://www.misprofesores.com/escuelas/BUAP-Be...,Puebla,Puebla,2516
973,Facultad de Contaduria y Administracion UNAM,https://www.misprofesores.com/escuelas/Faculta...,DF,DF,2356


Un objetivo es determinar cual asignatura tiene mayor dificultad o disparidad entre escuelas. Por lo que se necesita una base de datos que tenga cada profesor, su asignartura dada y la calificacion recibida en ella.

Como son muchas, comenzare haciendolo solo para facultad de ciencias de la UNAM: https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-UNAM_2842

* Obtener los datos de los profesores, su numero de rese;as, calificacion y url
  * Filtrar por quienes no tienen calificacion o rese;as
* Obtener de cada url y por cada comentario la materia, y la calificaciones y etiquetas por cada una.

In [ ]:
df_general.loc[df_general['Universidad']=='Facultad de Ciencias UNAM']

,Universidad,url_universidad,ciudad,estado,n_profesores
966,Facultad de Ciencias UNAM,https://www.misprofesores.com/escuelas/Faculta...,Ciudad de Mexico,Distrito Federal,3587
967,Facultad de Ciencias UNAM,https://www.misprofesores.com/escuelas/Faculta...,MEXICO,MEXICO,41


### Obtener url para la lista de profesores de la  facultad de ciencias, UNAM

In [ ]:
df_general['url_universidad'].loc[df_general['Universidad']=='Facultad de Ciencias UNAM'].iloc[0]

'https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-UNAM_2842'

### Obtener lista de profesores de la facultad de ciencias, UNAM

In [ ]:
url = 'https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-UNAM_28429'
response = requests.get(url)

if response.status_code == 200:
  soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup = soup.find_all('script', {'type':'text/javascript'})[3]

In [ ]:
soup = str(soup).split('dataSet')[1]

In [ ]:
soup = soup.split('=')[1]

In [ ]:
profesores = soup

In [ ]:
profesores = profesores.split(';\n$')[0]

In [ ]:
profesores

' [{"i":"17997","n":"SERGIO","a":"\\tCAMACHO MENDOZA ","d":"GESTION D RECURSOS GUBERNAM.","m":"7","c":"7.7143"},{"i":"75638","n":"rafael","a":"","d":"","m":"0","c":null},{"i":"15960","n":"Sosa Manzur","a":"Aalvar","d":"fcpys","m":"8","c":"9.0000"},{"i":"15959","n":"Sosa Manzur","a":"Aalvar","d":"filosof\\u00eda y teor\\u00eda pol\\u00edtica","m":"7","c":"6.4286"},{"i":"142833","n":"Oscar","a":"Abad","d":"Fin","m":"0","c":null},{"i":"105400","n":"Mario","a":"Abad Schoster","d":"FES Arag\\u00f3n","m":"1","c":"10.0000"},{"i":"110982","n":"Paulina","a":"Abarca Juarez","d":"Ciencias","m":"4","c":"5.0000"},{"i":"156013","n":"Estefania","a":"Abarca Rodriguez","d":"Facultad de Ingenier\\u00eda ","m":"2","c":"3.5000"},{"i":"141080","n":"Jose Sergio Celso","a":"Abarca Tellez","d":"Facultad de Contadur\\u00eda","m":"5","c":"2.2000"},{"i":"163856","n":"Ram\\u00f3n Severino","a":"Abascal Rivera","d":"Fes Zaragoza ","m":"0","c":null},{"i":"18104","n":"Luis Orlando ","a":"Abrajan Villase\\u00f1or","d

In [ ]:
profesores = json.loads(profesores)

In [ ]:
profesores = pd.DataFrame(profesores)

In [ ]:
profesores

,i,n,a,d,m,c
0,17997,SERGIO,\tCAMACHO MENDOZA,GESTION D RECURSOS GUBERNAM.,7,7.7143
1,75638,rafael,,,0,None
2,15960,Sosa Manzur,Aalvar,fcpys,8,9.0000
3,15959,Sosa Manzur,Aalvar,filosofía y teoría política,7,6.4286
4,142833,Oscar,Abad,Fin,0,None
...,...,...,...,...,...,...
3660,157238,Juana Silvia,Zúñiga Fonseca,Contaduría y administración,0,None
3661,150027,adriana,zúñiga pérez,fes acatlán francés,1,3.0000
3662,124905,Salvador,Zúñiga Prado,Latin,0,None
3663,162893,Maura,Zuñiga Rodriguez,Iztacala,1,10.0000


Ahora hay que crear los urls como muestra el programador de la pagina.
- Pero antes sera bueno eliminar las entradas que no sirven, es decir, las que no tienen calificacion o comentarios

In [ ]:
profesores.dtypes

i    object
n    object
a    object
d    object
m    object
c    object
dtype: object

In [ ]:
profesores['m'] = profesores['m'].astype(int)

In [ ]:
profesores = profesores.drop(profesores.loc[profesores['m']==0].index)

In [ ]:
profesores

,i,n,a,d,m,c
0,17997,SERGIO,\tCAMACHO MENDOZA,GESTION D RECURSOS GUBERNAM.,7,7.7143
2,15960,Sosa Manzur,Aalvar,fcpys,8,9.0000
3,15959,Sosa Manzur,Aalvar,filosofía y teoría política,7,6.4286
5,105400,Mario,Abad Schoster,FES Aragón,1,10.0000
6,110982,Paulina,Abarca Juarez,Ciencias,4,5.0000
...,...,...,...,...,...,...
3655,51903,Virgilio,Zuleta Contreras,Facultad de Economia /UNAM,26,6.3462
3659,102627,Sergio,Zúñiga Barrera,Ingeniería,3,2.0000
3661,150027,adriana,zúñiga pérez,fes acatlán francés,1,3.0000
3663,162893,Maura,Zuñiga Rodriguez,Iztacala,1,10.0000


Es notable como se repiten los nombres de muchos profesores. Esto se puede deber a que se crearon entradas diferentes segun la materia que impartia en su momento

### Obtener urls de cada profesor

In [ ]:
# Crear urls
def fix_url(nombre):
    nombre = nombre.strip()
    nombre = replace_latin_chars(nombre)
    nombre = re.sub(r'[^a-zA-Z0-9\s]', '', nombre)
    nombre = nombre.replace("(", "")
    nombre = nombre.replace(")", "")
    nombre = nombre.replace(" -", "-")
    nombre = nombre.replace("- ", "-")
    nombre = nombre.replace("\n", "")
    nombre = nombre.replace("\r", "")
    nombre = nombre.replace("  ", " ")
    nombre = nombre.replace("\\", "")
    nombre = nombre.replace(" ", "-")
    return nombre



In [ ]:
import re

def replace_latin_chars(s):
    translate_re = re.compile(r'[ÁÉÍÓÚÑÜáéíóúñü]')
    translate = {
        "Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U", "Ñ": "N", "Ü": "U",
        "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u", "ñ": "n", "ü": "u"
    }
    return translate_re.sub(lambda match: translate[match.group(0)], s)


In [ ]:
def profesor_url(nombre, apellido, id):
  base = 'https://www.misprofesores.com/profesores/'
  nombre = fix_url(nombre);
  apellido = fix_url(apellido);
  return base+nombre+"-"+apellido+"_"+id;

In [ ]:
url_profesores = []
for i in range(0, len(profesores)):
  url_profesores.append(profesor_url(profesores.iloc[i]['n'], profesores.iloc[i]['a'], profesores.iloc[i]['i']))

In [ ]:
profesores['url_profesor'] = url_profesores

In [ ]:
profesores

In [ ]:
profesores.to_csv('unam_profesores.csv', index=False)

In [ ]:
# Recibe la url de una universidad y devuelve una tabla con los profesores y sus URLs
def get_profesores(url):
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    soup = soup.find_all('script', {'type':'text/javascript'})[3]
    soup = str(soup).split('dataSet')[1]
    soup = soup.split('=')[1]
    temp = soup
    soup = soup.split(';\n$')[0]
    profesores = json.loads(soup)
    profesores = pd.DataFrame(profesores)

    profesores['m'] = profesores['m'].astype(int)
    profesores = profesores.drop(profesores.loc[profesores['m']==0].index)

    url_profesores = []
    for i in range(0, len(profesores)):
      url_profesores.append(profesor_url(profesores.iloc[i]['n'], profesores.iloc[i]['a'], profesores.iloc[i]['i']))
    profesores['url_profesor'] = url_profesores
  return profesores

In [ ]:
fc_unam_profes = get_profesores('https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-UNAM_2842')
# Funciona bien

In [ ]:
fc_unam_profes.head()

,i,n,a,d,m,c,url_profesor
0,151566,,,,1,2.0000,https://www.misprofesores.com/profesores/-_151566
1,84972,Actuaría,- Las chicas pesadas,Actuaría,16,7.5000,https://www.misprofesores.com/profesores/Actua...
2,84971,Física,- Los bob el constructor,Física,9,7.3333,https://www.misprofesores.com/profesores/Fisic...
3,84966,Actuaría,- Los don cangrejo,Actuaría,8,4.6250,https://www.misprofesores.com/profesores/Actua...
4,84964,Biología,- Los dora la exploradora,Biología,6,5.8333,https://www.misprofesores.com/profesores/Biolo...


## Obtener comentarios de los profesores

Vamos a obtener los datos de un profesor en especifico. El resultado deben ser dos tablas, una con sus etiquetas y otra con sus materias y calificaciones elegidas

In [ ]:
url = 'https://www.misprofesores.com/profesores/Victor-Romero-Rochin_41658'
# Un profesor excelente

### Obtener etiquetas principales

In [ ]:
def get_etiquetas(soup):
  #response = requests.get(url)

  # if response.status_code == 200:
  #   soup = BeautifulSoup(response.content, 'html.parser')

  calidad_general = soup.find('div', class_='breakdown-container quality').find('div', class_='grade').text
  lo_recomiendan = soup.find('div', class_="breakdown-section takeAgain 44").find('div', class_='grade').text
  dificultad = soup.find('div', class_="breakdown-section difficulty").find('div', class_='grade').text

  all_tags = soup.find('div', class_='tag-box')
  tags = all_tags.find_all('span', class_='tag-box-choosetags') 

  profesorID = []
  tag_profesor = []
  n_tags = []

  profesor = soup.find('title').text

  for tag in tags:
    tag = str(tag.text)
    profesorID.append(profesor)

    if '(' in tag:
      a = tag.find('(')
      b = tag.find(')')
      tag_profesor.append(tag[0:a])
      n_tags.append(int(tag[a+1:b]))
    else:
      tag_profesor.append(tag)
      n_tags.append(1)
    
    return pd.DataFrame({'Profesor_ID': profesorID,
                         'Etiquetas': tag_profesor,
                         'Cantidad': n_tags})

### Intervalos aleatorios

### Obtener calificaciones de comentarios

In [ ]:
from sqlalchemy import Null

In [ ]:
def get_datos(url):
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    pages = [url]
    links = soup.find('ul', class_='pagination').find_all('li')
    for link in links:
      if 'href' in str(link):
        pages.append(link.find('a').get('href'))
    
    url_id = url.replace('https://www.misprofesores.com/profesores/', '')
    # urls = urls[0:3]
    n = 0
    urls_id = []
    nombre = []
    fecha = []
    rating_type = []
    calidad_general = []
    facilidad = []
    materia = []
    interes = []
    c_recibida = []
    asistencia = []
    etiquetas = []
    comentario = []

    for page in pages:
      # Conectamos con la páginas
      time.sleep(random.uniform(0.2, 0.6))

      response = requests.get(page)
      soup = BeautifulSoup(response.content, 'html.parser')

      # Obtenemos la fila de la entrada
      rows = soup.find_all('tr')
      profesor = soup.find('title').text


      for row in rows:
        # Evitamos el encabezado de la tabla
        if 'head-label class' in str(row):
          None
        else:
          # Agregamos los datos a las listas

          #Nombre
          nombre.append(profesor)
          #Fecha
          if row.find('div', class_='date'):
            fecha.append(row.find('div', class_='date').text)
          else:
            fecha.append('No date')
          #Rating
          rating_type.append(row.find('span', class_='rating-type').text)
          #Calidad general
          calidad_general.append(row.find_all('div', class_='break')[0].find('span').text)
          #Facilidad
          facilidad.append(row.find_all('div', class_='break')[1].find('span').text)
          # Materia
          materia.append(row.find('td', class_='class').find('span', class_='name').find('span',class_='response').text)
          # Calificacion recibida
          if row.find('td', class_='class').find('span', class_='grade').find('span', class_='response'):
             c_recibida.append(row.find('td', class_='class').find('span', class_='grade').find('span', class_='response').text)
          else:
            c_recibida.append(None)
          # Asistencia
          if row.find('td', class_='class').find('span', class_='attendance'):
            asistencia.append(row.find('td', class_='class').find('span', class_='attendance').text)
          else:
            asistencia.append(None)
          # Interes
          if len(row.find('td', class_='class').find_all('span', class_= 'grade')) == 2:
            interes.append(row.find('td', class_='class').find_all('span', class_= 'grade')[1].find('span', class_='response').text)
          else:
            interes.append(None)
          #Etiquetas
          if row.find('div', class_='tagbox').find_all('span'):
            tags = []
            tagboxes = row.find('div', class_='tagbox').find_all('span')
            for tagbox in tagboxes:
              tags.append(tagbox.text)
            etiquetas.append(', '.join(tags))
          else:
            etiquetas.append(None)
          #Comentario
          if row.find('p', class_='commentsParagraph'):
            comentario.append(row.find('p', class_='commentsParagraph').text)
          else:
            comentario.append(None)
          urls_id.append(url_id)


  # print(f"Profesores : {len(nombre)} \n \
  #         Asignatura: {len(materia)} \n \
  #         Fecha: {len(fecha)} \n \
  #         Rating: {len(rating_type)} \n \
  #         Comentario: {len(comentario)} \n \
  #         Calidad: {len(calidad_general)} \n \
  #         Facilidad: {len(facilidad)} \n \
  #         Califacion: {len(c_recibida)} \n \
  #         Interes: {len(interes)} \n \
  #         Asistencia: {len(asistencia)} \n \
  #         Etiquetas: {len(etiquetas)}")

  df = pd.DataFrame({'Profesor': nombre,
                    'Asignatura': materia,
                    'Fecha': fecha,
                    'Rating': rating_type,
                    'Comentario': comentario,
                    'Calidad_general': calidad_general, 
                    'Facilidad':facilidad,
                    'Calificacion_recibida':c_recibida,
                    'Interes_clase': interes,
                    'Asistencia': asistencia,
                    'Etiquetas': etiquetas,
                    'url_id' : urls_id
                    })
  return df

In [ ]:
url = 'https://www.misprofesores.com/profesores/Victor-Romero-Rochin_41658'
df_datos = get_datos(url)

In [ ]:
df_datos.head()

,Profesor,Asignatura,Fecha,Rating,Comentario,Calidad_general,Facilidad,Calificacion_recibida,Interes_clase,Asistencia,Etiquetas,url_id
0,Victor Romero Rochin - Facultad de Ciencias UN...,Física Estadística,28/Ene/2023,BUENO,\n Uno de los mejores profe...,10.0,6.0,9,Medio,Asistencia: No obligatoria,"Inspiracional, Deja trabajos largos",Victor-Romero-Rochin_41658
1,Victor Romero Rochin - Facultad de Ciencias UN...,IFC,25/Ene/2023,BUENO,\n El profesor da un muy bu...,8.0,8.0,9,Alto,Asistencia: No obligatoria,"Muy cómico, Inspiracional, Tomaría su clase ot...",Victor-Romero-Rochin_41658
2,Victor Romero Rochin - Facultad de Ciencias UN...,Introducción a la Física Cuántica,21/Ene/2023,BUENO,\n Es un profesor que te tr...,9.0,6.0,9,Alto,Asistencia: Obligatoria,"Respetado por los estudiantes, Clases excelent...",Victor-Romero-Rochin_41658
3,Victor Romero Rochin - Facultad de Ciencias UN...,Introducción a la Física Cuántica,19/Ene/2023,BUENO,\n Buen curso. No es tan di...,9.0,6.0,9,Alto,Asistencia: No obligatoria,"Califica Duro, Clases excelentes, Tomaría su c...",Victor-Romero-Rochin_41658
4,Victor Romero Rochin - Facultad de Ciencias UN...,Termodinámica,05/Ene/2023,BUENO,\n De los profes top que tu...,8.0,2.0,7,Alto,Asistencia: No obligatoria,"Califica Duro, Deja trabajos largos, Los exáme...",Victor-Romero-Rochin_41658


Ahora es necesaria una función que extraiga los datos de cada url de los profesores y vaya uniendo las nuevas tablas con una general.

En caso de ser bloqueados, estaré guardando la tabla general con regularidad y creando un marcador para saber dónde continuar.

In [ ]:
cd drive/MyDrive/DA/Mis_profesores_project/

/content/drive/MyDrive/DA/Mis_profesores_project


In [ ]:
import os

fc_unam = 'https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-UNAM_2842'
df_fc_unam = get_profesores(fc_unam) 
fc_unam_url = df_fc_unam['url_profesor']
db_name = 'fc_unam_profes.csv'

if os.path.isfile(db_name):
  flag = 1
  all_df = pd.read_csv(db_name)
  print('Existe el archivo')
else:
  flag = 0
  print('No existe el archivo')
count = 0

for indice, url in enumerate(fc_unam_url):
  #print(url)

  url_id = url.replace('https://www.misprofesores.com/profesores/', '')
  
  if flag == 0:
    flag = 1
    df = get_datos(url)
    all_df = df
  else:
    if url_id not in list(all_df['url_id']):
      # df = get_datos(url)
      # all_df= pd.concat([all_df, df], axis=0, ignore_index=True)
      count += 1 
  if indice % 100 == 0 or indice == len(fc_unam_url) - 1:
    all_df.to_csv('fc_unam_profes.csv', index=False)
    print(f'{indice} profesores en la base de datos')

print(count)
  

Existe el archivo
0 profesores en la base de datos
100 profesores en la base de datos
200 profesores en la base de datos
300 profesores en la base de datos
400 profesores en la base de datos
500 profesores en la base de datos
600 profesores en la base de datos
700 profesores en la base de datos
800 profesores en la base de datos
900 profesores en la base de datos
1000 profesores en la base de datos
1100 profesores en la base de datos
1200 profesores en la base de datos
1300 profesores en la base de datos
1400 profesores en la base de datos
1500 profesores en la base de datos
1600 profesores en la base de datos
1700 profesores en la base de datos
1800 profesores en la base de datos
1900 profesores en la base de datos
2000 profesores en la base de datos
2100 profesores en la base de datos
2200 profesores en la base de datos
2300 profesores en la base de datos
2400 profesores en la base de datos
2500 profesores en la base de datos
2600 profesores en la base de datos
2700 profesores en la 

In [ ]:
len(all_df['url_id'].unique())

3202

In [ ]:
len(urls)

In [ ]:
fc_unam = 'https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-UNAM_2842'
df_fc_unam = get_profesores(fc_unam) 

In [ ]:
len(df_fc_unam)

3202